<h1> OpenAI API Key Setup



In [2]:
# LangChain 업데이트
# 1. Create a .env file in the directory.
# 2. Input the issued API key into the .env file.


In [4]:
from dotenv import load_dotenv

# API KEY load
load_dotenv()

False

In [6]:
import os

print(f"[API KEY]\n{os.environ['OPENAI_API_KEY'][:-15]}" + "*" * 15)

KeyError: 'OPENAI_API_KEY'

In [ ]:
from importlib.metadata import version

print("[LangChain 관련 패키지 버전]")
for package_name in [
    "langchain",
    "langchain-core",
    "langchain-experimental",
    "langchain-community",
    "langchain-openai",
    "langchain-huggingface",
    "langchain-google-genai",
    "langchain-anthropic",
    "langchain-cohere",
    "langchain-chroma",
    "langchain-elasticsearch",
    "langchain-upstage",
    "langchain-cohere",
    "langchain-milvus",
    "langchain-text-splitters",
]:
    try:
        package_version = version(package_name)
        print(f"{package_name}: {package_version}")
    except ImportError:
        print(f"{package_name}: 설치되지 않음")

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
# detemplate
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

<h1> LangChain Expression Langauge

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

<h1> LangChain Expression Langauge

In [ ]:
# Here, we use LCEL to combine various components into a single chain
# chain = prompt | model | output_parser

from langchain_core.output_parsers import StrOutputParser



In [ ]:
template = """
당신은 IMF의 10년차 경제학자입니다. 주어진 상황에 맞는 답을 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 답변:

"""

In [ ]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template(template)

model = ChatOpenAI()

output_parser = StrOutputParser()

# Chain (Connect prompt and model Objects with the Pipe (|) Operator)
chain = prompt | model | output_parser

In [ ]:
# Pass input Using the invoke Method

print(chain.invoke({"question": "지금 미국경제는 어떤가요"}))


In [ ]:
# When prompt is in 'yaml file'

from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml", encoding="utf-8")
prompt
print(prompt.format(country="대한민국"))

<h1> Few shot Prompt Template

In [14]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


examples = [
    {
        "question": "프리드먼와 케인즈 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 프리드먼은 몇 살에 사망했나요?
중간 답변: 프리드먼은 78세에 사망했습니다.
추가 질문: 케인즈는 몇 살에 사망했나요?
중간 답변: 케인즈는 76세에 사망했습니다.
최종 답변은: 프리드먼이 더 오래 살았습니다.
""",
    },
    {
        "question": "연준의장은 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 연준의장은 누구인가요?
중간 답변: 연준의장은 제롬 파월입니다.
추가 질문: 파월은 언제 태어났나요?
중간 답변: 파월은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
]

In [ ]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

print(example_prompt.format(**examples[0]))

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "연준의장은 언제 태어났나요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

<h1> Example Selector

In [ ]:
# When performing few-shot learning, having too many question examples increases token usage. 
# In such cases, an example selector is used to retrieve similar examples from a database.

In [ ]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Initialize the Chroma VectorStore with a name and embeddings
# Chroma serves as the vector database where the embeddings are stored.
chroma = Chroma("example_selector", OpenAIEmbeddings()) # OpenAIEmbeddings() is used to generate semantic embeddings for both the examples and the input query. 

# Create an example selector using Semantic Similarity
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # List of available examples
    examples,
    # Embedding class used to generate embeddings for semantic similarity
    OpenAIEmbeddings(),
    # VectorStore class used to store embeddings and perform similarity searches
    Chroma,
    # Number of examples to retrieve
    k=1,
)

question = "연준의장의 나이는 몇 살인가요?"

# Select most similiar example
selected_examples = example_selector.select_examples({"question": question})

print(f"입력에 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')